In [1]:
import torch
from torch import nn
from utils_2 import get_vocab, get_tags
import json
import nltk
from nltk import pos_tag, word_tokenize

# Download required NLTK data
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xAbdoMo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xAbdoMo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
def load_model_state(model, path):
    """
    Loads the model's parameters into a pre-defined architecture
    """
    model.load_state_dict(torch.load(path))
    model.eval()  # Set to evaluation mode
    device = torch.device("cuda:0")
    model.to(device)
    return model


In [16]:
vocab = get_vocab(['processed_input/train_vocab.txt', 'processed_input/dev_vocab.txt', 'processed_input/test_vocab.txt'])

tags_ct  = get_tags('processed_input/train_complex_topping_tags.txt')  # Complex Topping tags
tags_pz  = get_tags('processed_input/train_pizza_orders_tags.txt')     # Pizza orders tags
tags_dr  = get_tags('processed_input/train_drink_orders_tags.txt')     # Drink orders tags
tags_ob  = get_tags('processed_input/train_orders_tags.txt')           # Order boundary tags

print('Vocab size:', len(vocab))

Vocab size: 448


In [17]:
print(tags_ct)
print(tags_pz)
print(tags_dr)
print(tags_ob)

{'QUANTITY': 0, 'TOPPING': 1, 'TOPPING_S': 2, 'QUANTITY_S': 3, 'NONE': 4}
{'TOPPING_S': 0, 'COMPLEX_TOPPING_S': 1, 'NOT_STYLE_S': 2, 'STYLE': 3, 'NUMBER': 4, 'TOPPING': 5, 'SIZE': 6, 'NONE': 7, 'NOT_TOPPING': 8, 'NOT_TOPPING_S': 9, 'NUMBER_S': 10, 'NOT_COMPLEX_TOPPING_S': 11, 'STYLE_S': 12, 'NOT_COMPLEX_TOPPING': 13, 'COMPLEX_TOPPING': 14, 'SIZE_S': 15, 'NOT_STYLE': 16}
{'SIZE': 0, 'CONTAINERTYPE': 1, 'DRINKTYPE': 2, 'DRINKTYPE_S': 3, 'NONE': 4, 'CONTAINERTYPE_S': 5, 'VOLUME': 6, 'NUMBER': 7, 'VOLUME_S': 8, 'SIZE_S': 9, 'NUMBER_S': 10}
{'DRINKORDER': 0, 'DRINKORDER_S': 1, 'PIZZAORDER': 2, 'PIZZAORDER_S': 3, 'NONE': 4}


In [19]:
from ner import NER

model_boundary    = NER(embedding_dim=95, hidden_size=800, n_classes=len(tags_ob), vocab_size=len(vocab), num_layers=2, dropout=0.5)
model_pizza_order = NER(embedding_dim=95, hidden_size=1200, n_classes=len(tags_pz), vocab_size=len(vocab), num_layers=2, dropout=0.5)
model_drink_order = NER(embedding_dim=95, hidden_size=800, n_classes=len(tags_dr), vocab_size=len(vocab), num_layers=2, dropout=0.5)
model_complex     = NER(embedding_dim=95, hidden_size=800, n_classes=len(tags_ct), vocab_size=len(vocab), num_layers=1, dropout=0.5)

model_boundary = load_model_state(model_boundary, "models/order_boundary_e95_h800_l2_d0.5_x85.1.pth")
model_pizza_order = load_model_state(model_pizza_order, "models/pizza_order_e95_h1200_l2_d0.5_s1_x86.2.pth")
model_drink_order = load_model_state(model_drink_order, "models/drink_order_e95_h800_l2_d0.5_x94.8.pth")
model_complex = load_model_state(model_complex, "models/complex_e95_h800_l1_d0.5_x1.pth")

device = torch.device("cuda:0")
model_boundary.to(device)
model_pizza_order.to(device)
model_drink_order.to(device)
model_complex.to(device)

model_boundary.eval()
model_pizza_order.eval()
model_drink_order.eval()
model_complex.eval()

C:\Users\xAbdoMo\AppData\Local\Temp\ipykernel_12132\941207917.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))


NER(
  (embedding): Embedding(448, 95)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(95, 800, batch_first=True, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=1600, out_features=5, bias=True)
)

In [20]:
def tags_inverse(tags):
    inv_tags = {}
    for tag, value in tags.items():
        inv_tags[value] = tag
    return inv_tags

inv_tags_ct = tags_inverse(tags_ct)
inv_tags_pz = tags_inverse(tags_pz)
inv_tags_dr = tags_inverse(tags_dr)
inv_tags_ob = tags_inverse(tags_ob)

print(inv_tags_ct)
print(inv_tags_pz)
print(inv_tags_dr)
print(inv_tags_ob)

{0: 'QUANTITY', 1: 'TOPPING', 2: 'TOPPING_S', 3: 'QUANTITY_S', 4: 'NONE'}
{0: 'TOPPING_S', 1: 'COMPLEX_TOPPING_S', 2: 'NOT_STYLE_S', 3: 'STYLE', 4: 'NUMBER', 5: 'TOPPING', 6: 'SIZE', 7: 'NONE', 8: 'NOT_TOPPING', 9: 'NOT_TOPPING_S', 10: 'NUMBER_S', 11: 'NOT_COMPLEX_TOPPING_S', 12: 'STYLE_S', 13: 'NOT_COMPLEX_TOPPING', 14: 'COMPLEX_TOPPING', 15: 'SIZE_S', 16: 'NOT_STYLE'}
{0: 'SIZE', 1: 'CONTAINERTYPE', 2: 'DRINKTYPE', 3: 'DRINKTYPE_S', 4: 'NONE', 5: 'CONTAINERTYPE_S', 6: 'VOLUME', 7: 'NUMBER', 8: 'VOLUME_S', 9: 'SIZE_S', 10: 'NUMBER_S'}
{0: 'DRINKORDER', 1: 'DRINKORDER_S', 2: 'PIZZAORDER', 3: 'PIZZAORDER_S', 4: 'NONE'}


In [21]:
from utils_2 import tokenize, preprocess_tokens, project_tokens

def feed_model(model, query, inv_tags):
    s = tokenize(query)
    s = preprocess_tokens(s, 0)
    print(s)
    s = project_tokens(s, vocab)
    print(s)
    x_tensor = torch.tensor(s)
    with torch.no_grad():
        output = model.forward(x_tensor.to(device))
        output = torch.argmax(output, dim=-1).to("cpu")
        return [inv_tags[x.item()] for x in output]

feed_model(model_pizza_order, "i want to order a large pizza with pepperoni sausage black olives onions and anchovies", inv_tags_pz)    

['<pron>', 'want', 'to', 'order', 'a', 'larg', 'pizza', 'with', 'pepperoni', 'sausag', 'black', 'oliv', 'onion', 'and', 'anchovi']
[446, 211, 246, 227, 62, 207, 176, 69, 99, 100, 72, 74, 3, 209, 56]


['NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'TOPPING_S',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'TOPPING_S',
 'NONE',
 'NOT_COMPLEX_TOPPING_S']

In [46]:
def run_complex(order):
    words = [token for token in order.split(' ') if token != '']
    order_result = feed_model(model_complex, order, inv_tags_ct)
    # print(f"COMPLEX:\n\t IN -> {order}\n\t OUT -> {order_result}")
    result = ""
    index = 0
    
    TAGS_STARTERS = ["TOPPING_S", "QUANTITY_S"]
    TAGS_CONT     = ["TOPPING"  , "QUANTITY"  ]
    while index < len(order_result):  # len(order_result) == len(words)
        found = False
        for tag_s, tag in zip(TAGS_STARTERS, TAGS_CONT):
            if order_result[index] == tag_s:
                found = True
                content = [words[index]]
                index = index + 1
                while index < len(order_result) and order_result[index] == tag:
                    content = content + [words[index]]
                    index = index + 1
                result += f"({tag} {' '.join(content)} ) "
                break
        if not found:
            result += words[index] + " "
            index = index + 1
    
    return result

def run_pizza_order(order):
    words = [token for token in order.split(' ') if token != '']
    order_result = feed_model(model_pizza_order, order, inv_tags_pz)
    result = ""
    # print(f"PIZZA:\n\t IN -> {order}\n\t OUT -> {order_result}") 
    index = 0
    
    NORMAL_TAGS_STARTERS = ["TOPPING_S", "STYLE_S", "SIZE_S", "NUMBER_S"]
    NORMAL_TAGS_CONT     = ["TOPPING"  , "STYLE"  , "SIZE"  , "NUMBER"]
    
    NOT_TAGS_STARTERS    = ["NOT_TOPPING_S", "NOT_STYLE_S", "NOT_SIZE_S", "NOT_NUMBER_S"]   # last two doesn't exist but aahhh whatever xD
    NOT_TAGS_CONT        = ["NOT_TOPPING"  , "NOT_STYLE"  , "NOT_SIZE"  , "NOT_NUMBER"]     # I'll keep it just in case the model is tripping or something :)
    while index < len(order_result):  # len(order_result) == len(words)
        found = False
        for tag_s, tag in zip(NORMAL_TAGS_STARTERS, NORMAL_TAGS_CONT):
            if order_result[index] == tag_s:
                found = True
                content = [words[index]]
                index = index + 1
                while index < len(order_result) and order_result[index] == tag:
                    content = content + [words[index]]
                    index = index + 1
                if tag == "TOPPING":
                    for c in content:
                        result += f"({tag} {c} ) " # ' '.join(content)
                else:
                    result += f"({tag} {' '.join(content)} ) "
                break
        if found:
           continue
        
        for tag_s, tag in zip(NOT_TAGS_STARTERS, NOT_TAGS_CONT):
            if order_result[index] == tag_s:
                found = True
                content = [words[index]]
                index = index + 1
                while index < len(order_result) and order_result[index] == tag:
                    content = content + [words[index]]
                    index = index + 1
                result += f"(NOT ({tag[4:]} {' '.join(content)} ) ) "
                break
                
        if found:
           continue
            
        # special case: COMPLEX_TOPPING_S & NOT_COMPLEX_TOPPING_S
        if "COMPLEX_TOPPING_S" in order_result[index]:
            found = True
            negated = "NOT" in order_result[index]
            content = [words[index]]
            index = index + 1
            while index < len(order_result) and "COMPLEX_TOPPING" in order_result[index]:
                content = content + [words[index]]
                index = index + 1
            val = run_complex(' '.join(content))
            if negated:
                result += f"(NOT (COMPLEX_TOPPING {val}) ) "
            else:
                result += f"(COMPLEX_TOPPING {val}) "
        if found:
           continue
        result += words[index] + " "
        index = index + 1
    
    return result

def run_drink_order(order):
    words = [token for token in order.split(' ') if token != '']
    order_result = feed_model(model_drink_order, order, inv_tags_dr)
    # print(f"DRINK:\n\t IN -> {order}\n\t OUT -> {order_result}")
    result = ""
    index = 0
    
    TAGS_STARTERS = ["SIZE_S", "VOLUME_S", "NUMBER_S", "DRINKTYPE_S", "CONTAINERTYPE_S"]
    TAGS_CONT     = ["SIZE"  , "VOLUME"  , "NUMBER"  , "DRINKTYPE"  , "CONTAINERTYPE"]
    while index < len(order_result):  # len(order_result) == len(words)
        found = False
        for tag_s, tag in zip(TAGS_STARTERS, TAGS_CONT):
            if order_result[index] == tag_s:
                found = True
                content = [words[index]]
                index = index + 1
                while index < len(order_result) and order_result[index] == tag:
                    content = content + [words[index]]
                    index = index + 1
                result += f"({tag} {' '.join(content)} ) "
                break
        if not found:
            result += words[index] + " "
            index = index + 1
    
    return result

def run_order(order):
    words = [token for token in order.split(' ') if token != '']
    order_result = feed_model(model_boundary, order, inv_tags_ob)
    # print(f"ORDER:\n\t IN -> {order}\n\t OUT -> {order_result}")
    
    result = ""
    index = 0
    while index < len(order_result):  # len(order_result) == len(words)
        if order_result[index] in 'PIZZAORDER_S':  # read a pizza order
            order = [words[index]]
            index = index + 1
            while index < len(order_result) and order_result[index] == 'PIZZAORDER':
                order = order + [words[index]]
                index = index + 1
            if len(order) < 2: # pizza order must be at least two words
                result += ' '.join(order) + " "
            else:
                result += f"(PIZZAORDER {run_pizza_order(' '.join(order))}) "
        elif order_result[index] in 'DRINKORDER_S':  # read a drink order
            order = [words[index]]
            index = index + 1
            while index < len(order_result) and order_result[index] == 'DRINKORDER':
                order = order + [words[index]]
                index = index + 1
            if len(order) < 2: # drink order must be at least two words
                result += ' '.join(order) + " "
            else:
                result += f"(DRINKORDER {run_drink_order(' '.join(order))}) "
        else:
            result += words[index] + " "
            index = index + 1
    return result
    
def run_query(query):
    query = query.lower()
    return f"(ORDER {run_order(query)})"

In [47]:
result = run_query("we want some chicken pizza")
# expected = "(ORDER id like to order (PIZZAORDER (NUMBER two ) (SIZE large ) (STYLE gluten-free crust ) (TOPPING pepperoni ) pizzas ) with (DRINKORDER (NUMBER a ) (SIZE large ) (DRINKTYPE coke ) ) )"
print(f"Model   : {result}")
# print(f"Expected: {expected}")

Model   : (ORDER we want some (PIZZAORDER (TOPPING chicken ) pizza ) )


In [50]:
import json

def process_file(filename, prediction_function):
    total = 0
    correct = 0
    
    with open(filename, 'r') as file:
        for line in file:
            try:
                # Parse JSON object from each line
                data = json.loads(line.strip())
                
                # Extract source text and target parsing
                src = data['test.SRC']
                target = data['test.TOP']
                
                # Get prediction from the provided function
                prediction = prediction_function(src)
                
                # Compare prediction with target
                if prediction == target:
                    correct += 1
                else:
                    print(f"SRC     :  {src}")
                    print(f"EXPECTED:  {target}")
                    print(f"GOT     :  {prediction}")
                total += 1
                
            except json.JSONDecodeError:
                print(f"Warning: Skipping invalid JSON line: {line.strip()}")
            except KeyError as e:
                print(f"Warning: Missing required field: {e}")
                
    # Calculate accuracy
    accuracy = (correct / total) * 100 if total > 0 else 0
    return accuracy, correct, total


In [51]:
accuracy, correct, total = process_file("PIZZA_test.json", run_query)
    
print(f"Results:")
print(f"Total examples: {total}")
print(f"Correct predictions: {correct}")
print(f"Accuracy: {accuracy:.2f}%")

SRC     :  i want one medium pizza along with sausage mushrooms but hold ham please
EXPECTED:  (ORDER i want (PIZZAORDER (NUMBER one ) (SIZE medium ) pizza along with (TOPPING sausage ) (TOPPING mushrooms ) but hold (NOT (TOPPING ham ) ) ) please )
GOT     :  (ORDER i want (PIZZAORDER (NUMBER one ) (SIZE medium ) pizza along with (TOPPING sausage ) (TOPPING mushrooms ) (TOPPING but ) hold (NOT (TOPPING ham ) ) ) please )
SRC     :  i want to get a large extra cheese pepperoni pizza with no bacon
EXPECTED:  (ORDER i want to get (PIZZAORDER (NUMBER a ) (SIZE large ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) (TOPPING pepperoni ) pizza with no (NOT (TOPPING bacon ) ) ) )
GOT     :  (ORDER i want to get (PIZZAORDER (NUMBER a ) (SIZE large ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) pepperoni pizza with no (NOT (TOPPING bacon ) ) ) )
SRC     :  i need one large pie with onions thin crust chicken
EXPECTED:  (ORDER i need (PIZZAORDER (NUMBER one ) (SIZE large ) pie wi